In [1]:
import json
import matplotlib.pyplot as plt
import numpy as np
import warnings
import datetime
import argparse
import threading
import multiprocessing
import os

from tqdm import tqdm

from GBP_Simulations.GBP.data import DataGenerator
from GBP_Simulations.GBP.gbp import run_GaBP_SYNC_ACCELERATED, run_GaBP_SYNC_ACCELERATED_UPDATED, run_GaBP_HARDWARE_ACCELERATED, run_GaBP_HARDWARE_ACCELERATED_RESIDUAL_T_nP, run_GaBP_HARDWARE_ACCELERATED_RESIDUAL, run_GaBP_HARDWARE_ACCELERATED_RESIDUAL_NEW, run_GaBP_HARDWARE_ACCELERATED_RESIDUAL_T_MAX
from GBP_Simulations.GBP.utilities import HiddenPrints
from GBP_Simulations.GBP.visulisation import set_plot_options, get_plot_colors, NetworkxGraph, AnalyzeResult

# Option 1: Suppress all warnings
warnings.filterwarnings("ignore")

set_plot_options()
colors = get_plot_colors()

data_gen = DataGenerator()
result_analyzer = AnalyzeResult()

### 1D Line

In [2]:
# A, b = data_gen.get_1D_line_matrix_PSD(1000)

x = 32
y = 32

A,b = data_gen.get_2D_lattice_matrix_PSD(x,y,scaling=False)

In [3]:
b_prime = b
b_prime = b * 10 / 3.5402822502798086e-05

In [4]:
sync_convergence_threshold = 1*10**-0 # convergence threshold
convergence_type = 'all' #all
simulation_convergence_threshold = 1*10**-10

P_i, mu_i, N_i, P_ii, mu_ii, P_ij, mu_ij, iter_dist, stand_divs, means, iteration = run_GaBP_SYNC_ACCELERATED(A, 
                                                                                                      b_prime, 
                                                                                                      max_iter=1_000, 
                                                                                                      mae=True if convergence_type == 'mae' else False, 
                                                                                                      convergence_threshold=simulation_convergence_threshold,
                                                                                                      show=True)
final_mean_sync = list(mu_i)

iteration: 1
14.334075492125768
-----
iteration: 2
0.4078132602836402
-----
iteration: 3
0.06460986467816414
-----
iteration: 4
0.010000008799523352
-----
iteration: 5
0.001672342226769203
-----
iteration: 6
0.0003293966014785403
-----
iteration: 7
6.459163700284941e-05
-----
iteration: 8
1.2654165754047011e-05
-----
iteration: 9
2.5788912359218523e-06
-----
iteration: 10
4.1295309247228556e-07
-----
iteration: 11
0.0
-----


In [5]:
print(A.shape)

(1024, 1024)


### Hardware

In [15]:
ASYNC_ITER = 1
NODE_UPDT_PE = 192
PEs = 4

async_convergence_threshold = 1*10**-5

In [16]:
sum_of_iterations = 0
num_iterations = ASYNC_ITER

for it in range(0,num_iterations):
    # print(f"-------------- ITERATION = {it+1} --------------")
    P_i, mu_i, iteration = run_GaBP_HARDWARE_ACCELERATED(A, b_prime, caching=False, node_updates_per_pe=NODE_UPDT_PE, number_pes=PEs, TRUE_MEAN=final_mean_sync, max_iter=1000, mae=True, convergence_threshold=async_convergence_threshold, mode='random', show=True)
    sum_of_iterations += iteration
    print(f"-------------- ITERATION = {it+1} => Streams = {iteration} --------------")

print(f"AVE. ASYNC ITERATIONS = {sum_of_iterations/num_iterations}")

iteration: 0
9.718723946162072
-----
iteration: 1
9.718723946162072
-----
iteration: 2
2.651041868918502
-----
iteration: 3
0.9477670287535876
-----
iteration: 4
0.35049555058029447
-----
iteration: 5
0.06976525016983068
-----
iteration: 6
0.021844033487901374
-----
iteration: 7
0.007224629341408097
-----
iteration: 8
0.0017658473954421385
-----
iteration: 9
0.00038326668811028175
-----
iteration: 10
0.00010806762919183231
-----
iteration: 11
4.4121460470397536e-05
-----
iteration: 12
1.646454822236366e-05
-----
iteration: 13
3.504470993450688e-06
-----
-------------- ITERATION = 1 => Streams = 14 --------------
AVE. ASYNC ITERATIONS = 14.0


In [20]:
sum_of_iterations = 0
num_iterations = ASYNC_ITER

for it in range(0,num_iterations):
    # print(f"-------------- ITERATION = {it+1} --------------")
    # _, _, iteration, _, _ = run_GaBP_HARDWARE_ACCELERATED_RESIDUAL_T_nP(A, b_prime, caching=True, node_updates_per_pe=NODE_UPDT_PE, number_pes=PEs, TRUE_MEAN=final_mean_sync, max_iter=1000, mae=True, convergence_threshold=async_convergence_threshold, show=True)
    _, _, iteration, _, _ = run_GaBP_HARDWARE_ACCELERATED_RESIDUAL_T_MAX(A, b_prime, caching=True, node_updates_per_pe=NODE_UPDT_PE, number_pes=PEs, TRUE_MEAN=final_mean_sync, max_iter=1000, mae=True, convergence_threshold=async_convergence_threshold, show=True)

    sum_of_iterations += iteration
    print(f"-------------- ITERATION = {it+1} => Streams = {iteration} --------------")

print(f"AVE. ASYNC ITERATIONS = {sum_of_iterations/num_iterations}")

iteration: 1
2.6333945785162944
-----
iteration: 2
0.9847323824004739
-----
iteration: 3
0.42494725401057437
-----
iteration: 4
0.16642374704100235
-----
iteration: 5
0.10921897823097404
-----
iteration: 6
0.07270284145080441
-----
iteration: 7
0.05168830093149803
-----
iteration: 8
0.036942866222840286
-----
iteration: 9
0.02580333561005874
-----
iteration: 10
0.022454168430052233
-----
iteration: 11
0.01812565620405753
-----
iteration: 12
0.016070129418418448
-----
iteration: 13
0.014043983696007364
-----
iteration: 14
0.012278091242436187
-----
iteration: 15
0.00895715410541517
-----
iteration: 16
0.007799689289049115
-----
iteration: 17
0.006495138524979288
-----
iteration: 18
0.004612730683842409
-----
iteration: 19
0.0034845144454152144
-----
iteration: 20
0.002732438496800869
-----
iteration: 21
0.0024080674710453965
-----
iteration: 22
0.002026582276501849
-----
iteration: 23
0.001757502631076493
-----
iteration: 24
0.0016019301863265422
-----
iteration: 25
0.001511470270449179

In [19]:
sum_of_iterations = 0
num_iterations = ASYNC_ITER

for it in range(0,num_iterations):
    # print(f"-------------- ITERATION = {it+1} --------------")
    # _, _, iteration, _, _ = run_GaBP_HARDWARE_ACCELERATED_RESIDUAL_T_nP(A, b_prime, caching=True, node_updates_per_pe=NODE_UPDT_PE, number_pes=PEs, TRUE_MEAN=final_mean_sync, max_iter=1000, mae=True, convergence_threshold=async_convergence_threshold, show=True)
    _, _, iteration = run_GaBP_HARDWARE_ACCELERATED_RESIDUAL_NEW(A, b_prime, caching=True, node_updates_per_pe=NODE_UPDT_PE, number_pes=PEs, TRUE_MEAN=final_mean_sync, max_iter=1000, mae=True, convergence_threshold=async_convergence_threshold, show=True)

    sum_of_iterations += iteration
    print(f"-------------- ITERATION = {it+1} => Streams = {iteration} --------------")

print(f"AVE. ASYNC ITERATIONS = {sum_of_iterations/num_iterations}")

iteration: 1
2.6333945785162944
-----
iteration: 2
0.9977365199765678
-----
iteration: 3
0.4022417718171285
-----
iteration: 4
0.40199788739002806
-----
iteration: 5
0.31861009757096925
-----
iteration: 6
0.2694707619411603
-----
iteration: 7
0.20160858974342866
-----
iteration: 8
0.19589968931807863
-----
iteration: 9
0.1891168779986728
-----
iteration: 10
0.16751181661127634
-----
iteration: 11
0.14975063194584254
-----
iteration: 12
0.14322192525206406
-----
iteration: 13
0.10646758279154905
-----
iteration: 14
0.10608108078977935
-----
iteration: 15
0.10203232657218157
-----
iteration: 16
0.08576308870132687
-----
iteration: 17
0.08286571779788582
-----
iteration: 18
0.08202922974453981
-----
iteration: 19
0.07649063808104559
-----
iteration: 20
0.07386231898739022
-----
iteration: 21
0.07273175105529553
-----
iteration: 22
0.06907355160069562
-----
iteration: 23
0.06216453500133384
-----
iteration: 24
0.05510431089833249
-----
iteration: 25
0.053056322303085324
-----
iteration: 26